In [3]:
%load_ext autoreload
%autoreload 2

import os, re, json, ast
from llm_api_calls import send_message_to_gemini_async
from km_utils import get_ToCs, split_dict_by_size, get_content_from_articles_response, split_list_by_size
from coap_prompts import *
from parallel_descriptions import RateLimiter
from pprint import pprint

# rate_limiter = RateLimiter(9, 10)
rate_limiter = RateLimiter(13, 14)
# rate_limiter = RateLimiter(1, 10)

# Pipeline

In [4]:
ToCs = get_ToCs(base_path = 'coap_map')

doc_with_childs = ToCs['doc_with_childs']
ToC_sections = ToCs['ToC_sections']
ToC_chapters = ToCs['ToC_chapters']
ToC_chapters_with_article_short_descriptions = ToCs['ToC_chapters_with_article_short_descriptions']
ToC_articles = ToCs['ToC_articles']
ToC_articles_short_descriptions = ToCs['ToC_articles_short_descriptions']
ToC_articles_descriptions = ToCs['ToC_articles_descriptions']
ToC_sections_articles_shortDescription = ToCs['ToC_sections_articles_shortDescription']

In [45]:
user_query_ori = "Что будет за марихуану?"

async def pipline_with_3_steps(test_case, rate_limiter=None):
    user_query_ori = test_case['user_query_ori']

    user_query = await get_perefrased_query(user_query_ori, rate_limiter=rate_limiter)
    user_query = user_query['text_response']
    print(f'Изначальный вопрос: {user_query_ori}')
    print(f'Уточняющие вопросы: {user_query}')

    get_section_response = await get_section_nagigation(user_query_ori, user_query, ToC_sections, rate_limiter=rate_limiter)
    choosed_sections = parse_section_response(get_section_response, ToC_articles)

    total_articles = []
    for section in choosed_sections:
        print('section:', section)
        ToC_article_chunks = split_list_by_size(list(ToC_sections_articles_shortDescription[section].items()), size_limit=25000)
        choosed_articles_responses = await get_chunked_ARTICLE_navigation(user_query_ori, user_query, ToC_article_chunks, rate_limiter)
        articles = get_content_from_articles_response(ToC_articles, choosed_articles_responses)
        print(f'Найдено {len(articles)} статей')
        total_articles.extend(articles)

    print(f'Найдено всего {len(total_articles)} статей')
    print('***')
    pprint([a['article_title'] for a in total_articles], width=300)
    print('***')

    context = [f"{x['article_title']} : {x['content']}" for x in total_articles]
    if len(str(context)) > 25000:
        context = choose_articles_second_round_and_limit_context(context, user_query_ori, user_query, ToC_articles, size_limit=25000, rate_limiter=rate_limiter)

    get_article_response, prompt = await get_final_response(user_query_ori, user_query, 
                                                            context,
                                                            rate_limiter=rate_limiter)
    print('\n\n ФИНАЛЬНЫЙ ОТВЕТ: ***')
    print(get_article_response.get('text_response'))
    test_case['extracted_articles'] = total_articles
    test_case['final_response'] = get_article_response.get('text_response')
    test_case['user_query_perephrased'] = user_query

from test_cases import test_cases

tasks = []
for ind, test_case in enumerate(test_cases):
    tasks.append(pipline_with_3_steps(test_case, rate_limiter=RateLimiter(20, 25)))

await asyncio.gather(*tasks)
test_cases

Error 429 Resource has been exhausted (e.g. check quota)., retrying in 1 seconds... (Attempt 1/10)
Error 429 Resource has been exhausted (e.g. check quota)., retrying in 1 seconds... (Attempt 1/10)
Error 429 Resource has been exhausted (e.g. check quota)., retrying in 1 seconds... (Attempt 1/10)
Error 429 Resource has been exhausted (e.g. check quota)., retrying in 1 seconds... (Attempt 1/10)
Изначальный вопрос: какое наказание за марихуану?
Уточняющие вопросы: ["Какова мера наказания за хранение наркотических средств в административном порядке?", "В каком размере предусмотрен административный штраф за хранение небольшой партии каннабиса?", "Какая статья КОАП РФ регламентирует ответственность за незаконный оборот наркотиков?"]
Изначальный вопрос: какой штраф за порчу чужого имущества?
Уточняющие вопросы: ["Какой размер штрафа предусмотрен за умышленное повреждение или уничтожение чужого имущества?", "Какова мера наказания за порчу чужого имущества по КоАП РФ?"]
Error 429 Resource has b

/opt/homebrew/Caskroom/miniconda/base/lib/python3.11/asyncio/events.py:84: RuntimeWarning: coroutine 'choose_articles_second_round_and_limit_context' was never awaited
  self._context.run(self._callback, *self._args)


Error 429 Resource has been exhausted (e.g. check quota)., retrying in 128 seconds... (Attempt 8/10)
Error 429 Resource has been exhausted (e.g. check quota)., retrying in 128 seconds... (Attempt 8/10)
Error 429 Resource has been exhausted (e.g. check quota)., retrying in 128 seconds... (Attempt 8/10)
Error 429 Resource has been exhausted (e.g. check quota)., retrying in 128 seconds... (Attempt 8/10)
Error 429 Resource has been exhausted (e.g. check quota)., retrying in 128 seconds... (Attempt 8/10)
Error 429 Resource has been exhausted (e.g. check quota)., retrying in 128 seconds... (Attempt 8/10)
Error 429 Resource has been exhausted (e.g. check quota)., retrying in 128 seconds... (Attempt 8/10)
Error 429 Resource has been exhausted (e.g. check quota)., retrying in 128 seconds... (Attempt 8/10)
Error 429 Resource has been exhausted (e.g. check quota)., retrying in 128 seconds... (Attempt 8/10)
Error 429 Resource has been exhausted (e.g. check quota)., retrying in 128 seconds... (Atte

[{'user_query_ori': 'что будет за дискредитацию армии рф?',
  'needed_articles': ['Статья 20.3.3.'],
  'extracted_articles': [{'article_id': 'Статья 20.3.3.',
    'article_title': 'Статья 20.3.3. Публичные действия, направленные на дискредитацию использования Вооруженных Сил Российской Федерации в целях защиты интересов Российской Федерации и ее граждан, поддержания международного мира и безопасности или исполнения государствен',
    'content': '1. Публичные действия, направленные на дискредитацию использования Вооруженных Сил Российской Федерации в целях защиты интересов Российской Федерации и ее граждан, поддержания международного мира и безопасности, в том числе публичные призывы к воспрепятствованию использования Вооруженных Сил Российской Федерации в указанных целях, а равно направленные на дискредитацию исполнения государственными органами Российской Федерации своих полномочий за пределами территории Российской Федерации в указанных целях, если эти действия не содержат признаков 

In [46]:
def calculate_metrics(test_cases, k=10):
    mrr_sum = 0
    total_recall_at_k = 0
    for test_case in test_cases:
        needed_articles = set(test_case['needed_articles'])
        retrieved_articles_positions = {ra['article']: ra['position'] for ra in test_case.get('retrieved_articles', [])}
        
        # MRR calculation
        ranks = [1/position for article, position in retrieved_articles_positions.items() if article in needed_articles]
        if ranks:
            mrr_sum += max(ranks)  # Only the first occurrence of each needed article contributes to MRR
        
        # Recall at K calculation
        hits_at_k = sum(1 for article, position in retrieved_articles_positions.items() if article in needed_articles and position <= k)
        total_recall_at_k += hits_at_k / len(needed_articles) if needed_articles else 0


    mrr = mrr_sum / len(test_cases)
    recall_at_k = total_recall_at_k / len(test_cases)
    return {'mrr':mrr, 'recall_at_k':recall_at_k}

for test_case in test_cases:
    test_case['retrieved_articles'] = []
    for position, extracted_article in enumerate(test_case['extracted_articles'], start=1):
        retrieved_articles = [{'article': extracted_article['article_id'], 'position': position}]
        test_case['retrieved_articles'].extend(retrieved_articles)

calculate_metrics(test_cases, k=10)

{'mrr': 0.8240740740740741, 'recall_at_k': 0.9444444444444444}

In [51]:
for case in test_cases:
    pprint(case, width=200)

{'extracted_articles': [{'article_id': 'Статья 20.3.3.',
                         'article_title': 'Статья 20.3.3. Публичные действия, направленные на дискредитацию использования Вооруженных Сил Российской Федерации в целях защиты интересов Российской '
                                          'Федерации и ее граждан, поддержания международного мира и безопасности или исполнения государствен',
                         'content': '1. Публичные действия, направленные на дискредитацию использования Вооруженных Сил Российской Федерации в целях защиты интересов Российской Федерации и ее '
                                    'граждан, поддержания международного мира и безопасности, в том числе публичные призывы к воспрепятствованию использования Вооруженных Сил Российской Федерации в '
                                    'указанных целях, а равно направленные на дискредитацию исполнения государственными органами Российской Федерации своих полномочий за пределами территории '
               

In [1]:
# function to go over all directories recursively and change folder name to max of 230 charakters


import os
from pathlib import Path

def rename_directories(root_path):
    for path, dirs, files in os.walk(root_path, topdown=False):
        for name in dirs:
            full_path = os.path.join(path, name)
            if len(name) > 230:
                # Trim the folder name to 230 characters
                new_name = name[:230]
                new_full_path = os.path.join(path, new_name)
                # Make sure the new path does not already exist
                if not os.path.exists(new_full_path):
                    os.rename(full_path, new_full_path)
                    print(f"Renamed: {full_path} to {new_full_path}")
                else:
                    print(f"Cannot rename: {full_path} because {new_full_path} already exists")

# Example usage
root_directory = "coap_map"
rename_directories(root_directory)

Renamed: coap_map/Раздел II. Особенная часть/Глава 12. Административные правонарушения в области дорожного движения/Статья 12.21.3. Несоблюдение требований законодательства Российской Федерации о внесении платы в счет возмещения вреда, причиняемого автомобильным дорогам общего пользования федерального значения транспортными средствами, имеющими разрешенную максима to coap_map/Раздел II. Особенная часть/Глава 12. Административные правонарушения в области дорожного движения/Статья 12.21.3. Несоблюдение требований законодательства Российской Федерации о внесении платы в счет возмещения вреда, причиняемого автомобильным дорогам общего пользования федерального значения транспортными средствами, имеющими
Renamed: coap_map/Раздел II. Особенная часть/Глава 12. Административные правонарушения в области дорожного движения/Статья 12.5. Управление транспортным средством при наличии неисправностей или условий, при которых эксплуатация транспортных средств запрещена, или транспортным средством, на к